<a href="https://colab.research.google.com/github/CarolBw/Mundo_Machine_Learn/blob/main/Aula_01_atividades_perseptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementando um Perceptron

In [1]:
import numpy as np  # Importamos a biblioteca NumPy para realizar operações numéricas e vetorizadas.
from sklearn.metrics import accuracy_score  # Importamos a função de cálculo de acurácia da biblioteca scikit-learn.

# Criamos classe Perceptron com parâmetros configuráveis.
class Perceptron:
    def __init__(self, tx_aprendizado=0.1, epocas=500, early_stopping=False, tolerance=0.01, patience=10):
        self.tx_aprendizado = tx_aprendizado  # Taxa de aprendizado controla o tamanho do passo na atualização dos pesos.
        self.epocas = epocas  # Número de passagens completas pelo dataset.
        self.pesos = None  # Inicialmente os pesos são None, serão inicializados no método fit.
        self.bias = None  # Bias é um termo constante adicionado à decisão.
        self.early_stopping = early_stopping  # Se True, permite parar o treinamento cedo se não houver melhora.
        self.tolerance = tolerance  # Define quanto de melhora é necessária para considerar que ainda há progresso.
        self.patience = patience  # Quantas épocas sem melhora são toleradas antes de parar.

    # Função de ativação do tipo degrau, que decide a classe baseada no sinal da entrada.
    def funcao_ativacao(self, x):
        return np.where(x >= 0, 1, 0)

    # Método de treinamento do Perceptron. Recebe os dados (x) e as etiquetas/labels (y).
    def fit(self, x, y):
        n_samples, n_features = x.shape  # Número de amostras(linhas) e características(colunas) dos dados.
        self.pesos = np.random.rand(n_features)  # Inicializa pesos aleatoriamente.
        self.bias = 0  # Inicializa o bias como 0.
        no_improve_epoch = 0  # Contador para épocas sem melhoria.
        last_loss = np.inf  # Guarda o último valor da função de perda.

        for epoca in range(self.epocas):
            saida_linear = np.dot(x, self.pesos) + self.bias  # Calcula saída linear.
            previsao_y = self.funcao_ativacao(saida_linear)  # Aplica função de ativação.
            erros = y - previsao_y  # Calcula erro entre previsão e valor real.
            self.pesos += self.tx_aprendizado * np.dot(x.T, erros)  # Atualiza pesos.
            self.bias += self.tx_aprendizado * np.sum(erros)  # Atualiza bias.

            # Lógica de early stopping para evitar overfitting.
            current_loss = np.mean(erros ** 2)
            if self.early_stopping:
                if last_loss - current_loss < self.tolerance:
                    no_improve_epoch += 1
                else:
                    no_improve_epoch = 0
                if no_improve_epoch > self.patience:
                    print(f"Stopping early at epoch {epoca}")
                    break
                last_loss = current_loss

    # Método para prever classes baseadas em novos dados de entrada (x).
    def predict(self, x):
        saida_linear = np.dot(x, self.pesos) + self.bias  # Calcula saída linear.
        return self.funcao_ativacao(saida_linear)  # Retorna a classificação pela função de ativação.

    # Método para calcular a acurácia do modelo comparando previsões com verdadeiros rótulos (y).
    def score(self, x, y):
        y_pred = self.predict(x)  # Obtém previsões para os dados fornecidos.
        return accuracy_score(y, y_pred)  # Calcula e retorna a acurácia comparando previsões e rótulos verdadeiros.


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importando um Dataset da biblioteca para testar o perceptron
data = load_breast_cancer()

# Separando caracteristicas e rotulos nas variaveis x e y
x = data.data
y = data.target

# Dividir os dados em treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalizar os dados para melhor desempenho do Perceptron
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [3]:
# Criar uma instância do Perceptron
perceptron = Perceptron(tx_aprendizado=0.2, epocas=1100, early_stopping=True, tolerance=0.0001, patience=20)

# Treinar o Perceptron
perceptron.fit(x_train, y_train)

# Avaliar o desempenho do modelo
accuracy = perceptron.score(x_test, y_test)
print(f"Acurácia do Perceptron: {accuracy:.2f}")



Stopping early at epoch 983
Acurácia do Perceptron: 0.95
